# xBOUT

## Readable and scalable BOUT++ data analysis in python

Tom Nicholas, John Omotani & Rhys Doyle

(thomas.nicholas@york.ac.uk)

# Outline

- Multidimensional data analysis
- What is xarray?
- numpy vs xarray - syntax comparison 
- xarray basic features
- xarray + dask
- xBOUT

# Multidimensional data analysis

Most plasma physicists have some similar data analysis requirements:

- Relatively large datasets (up to 100's GBs)
- Multidimensional
- Warped-grid
- Several plasma quantities (density, temperature, ...)
- Lots of metadata to attach (simulation input file, shot number ...)
- Visualise multiple dimensions easily
- Apply mathematical operations over many dimensions easily and clearly

# What is xarray?

xarray is an open-source python library which aims to provide Pandas-like labelling, visualisation & analysis functionality for N-dimensional data.

Developed by atmospheric physicists, who have similar data analysis needs to us.

They also have relatively large, multidimensional, warped-grid, fluid turbulence datasets to visualise and analyse.

Sponsored by NumFocus, who also support NumPy, MatPlotLib, Pandas, Jupyter, IPython, Julia...

Large open-source project: github says used by 1.5k other packages

### xarray basic features: 
- Labelled multidimensional data
- Clear syntax for operations
- Lazy loading into memory
- Plotting convenience

# xarray: Labelled multidimensional data

xarray wraps numpy arrays with their `dims` as `DataArray`s.

Can also select data via special variables called `coords`.

Coordinates can be multidimensional
(e.g. for mapping Orthogonal toroidal coordinates -> field-aligned coordinates)

Multiple `DataArrays` are stored in same `Dataset`.

For example, imagine we had some output from an atmospheric fluid simulation...

<img src="dataset-diagram.png">

In [ ]:
<xarray.Dataset>
Dimensions:        (t: 8, x: 8, y: 8)
Coordinates:
  * t              (t) int64 0 1 2 3 4 5 6 7 8
  * x              (x) int64 0 1 2 3 4 5 6 7 8
  * y              (y) int64 0 1 2 3 4 5 6 7 8
    latitude       (x, y) float32 numpy.array(8, 8)
    longitude      (x, y) float32 numpy.array(8, 8)
Data variables:
    temperature    (t, x, y) float32 numpy.array(8, 8, 8)
    precipitation  (t, x, y) float32 numpy.array(8, 8, 8)
    
Attributes:
    reference_time 123.0
    input          {'solver': {'mxstep': 100000000, 'type': 'cvode'}, 'timeste...

Also carries around a dictionary of "attributes". I use that to store units, normalisations etc, as well as to store my entire input file for my simulation.

# numpy vs xarray: Clearer syntax for typical operations

Problem: We have some data $n(t,x,y,z)$, and we want to find the maximum over time of the spatially-averaged density at the separatrix. 

i.e. find $\text{max}(<n(t,x=\text{separatrix})>)$, where $<...>$ is an average over $y$ & $z$: 

Bare numpy:

In [ ]:
max_separatrix_density = np.max(np.mean(n[:, sep_x, ...], axis=(2,3)), axis=0)

xarray:

In [ ]:
max_separatrix_density = ds['n'].isel(x=sep_x).mean(dim=('y', 'z')).max(dim='t')

The xarray code is **clearer**, more **general**, contains **less magic** numbers, and the order of operations applied reads **left-to-right**.

# Copycats in other languages

Similar libraries implemented in at least 2 other languages, who credit inspiration to `xarray`.

C++ - [xframe](https://github.com/QuantStack/xframe)

Julia - [AxisArrays.jl](https://github.com/JuliaArrays/AxisArrays.jl)

# xarray: Lazy loading into memory

xarray uses the netCDF format in the backend.

Lazily loads data values - never waste RAM on unneeded values.

In [ ]:
import xarray as xr

# Open a 100GB file
ds = xr.open_dataset('BOUT_data.nc')

# Select a 1GB subset of the data
data = ds.isel(y=0)

# Data is only loaded into memory here, when we actually need it
result = some_maths(data)

# xarray: Plotting convenience

xarray provides plotting functions which wrap matplotlib.

In [ ]:
data_slice = ds['phi'].isel(y=34, t=-1)

data_slice.plot()

<img src="phi_2D_y=34.png" alt="Drawing" style="width: 600px;">

These plotting functions automatically use an appropriate type of plot for the dimension of the data (1D, 2D etc.)

In [ ]:
data = ds['T'].mean(dim=('t', 'y', 'z'))

fig, ax = plt.subplots()
data.plot.line(ax=ax)

plot_separatrix(data, sep_position, ax=ax)

<img src="T_profile.png" alt="Drawing" style="width: 600px;">

# xarray + dask: Memory chunking

If you also install the dask library (literally that's it, no other compiling or anything required), xarray will provide the option to load data into memory in chunks.

In [ ]:
ds = xr.open_dataset('example-data.nc', chunks={'time': 10})

print(ds)

In [ ]:
<xarray.Dataset>
Dimensions:      (latitude: 180, longitude: 360, time: 365)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01 2015-01-02 2015-01-03 ...
  * longitude    (longitude) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
  * latitude     (latitude) float64 89.5 88.5 87.5 86.5 85.5 84.5 83.5 82.5 ...
Data variables:
    temperature  (time, latitude, longitude) float64 dask.array<shape=(365, 180, 360), chunksize=(10, 180, 360)>

Now if you apply any xarray or any "embarrassingly parallel" numpy function to this dataset then it will compute the result only on one chunk at a time, and combine the results at the end.

This is very useful when you have "medium data": data larger than your system's RAM but not "big data". 

# xarray + dask: Parallel analysis

dask can also automatically parallelize the operation of any xarray function, and most numpy and scipy functions, using the ``apply_ufunc`` helper function: 

In [ ]:
result = xr.apply_ufunc(some_numpy_analysis_fn, ds, dask='parallelized', output=[float])

Chunking and parallelization through dask integration should allow you to easily scale up whatever analysis you were doing with numpy to work on datasets that are 100's of GBs in size.

In [4]:
import xarray as xr

squared_error = lambda x, y: (x - y) ** 2

arr1 = xr.DataArray([0, 1, 2, 3], dims='x')

xr.apply_ufunc(squared_error, arr1, 1)

<xarray.DataArray (x: 4)>
array([1, 0, 1, 4])
Dimensions without coordinates: x

In [7]:
import numpy as np

def vector_norm(x, dim, ord=None):
    return xr.apply_ufunc(np.linalg.norm, x,
                          input_core_dims=[[dim]],
                          kwargs={'ord': ord, 'axis': -1})

In [8]:
vector_norm(arr1, dim='x')

<xarray.DataArray ()>
array(3.741657)

# How does dask work?

<img src="collections-schedulers.png">

Dask works by:
    
- Labelling the various operations you want to perform, using either dask objects like dask.arrays or encoding general functions using dask.delayed

- Instead of evaluating these operations, it organises them into a Task Graph for later evaluation

- Evaluates them using one of a set of Schedulers, which can perform in parallel.

In [8]:
def inc(x):
    return x + 1

def add(x, y):
    return x + y

In [9]:
import dask

x = dask.delayed(inc)(1)
y = dask.delayed(inc)(2)
z = dask.delayed(add)(x, y)
z.compute()

5

In [ ]:
z.vizualize()

<img src="./inc-add.svg">

# xBOUT

### Loading data

xBOUT allows you to load all your BOUT++ output into an xarray dataset.

(parallel output of BOUT meant this wasn't simple: required significant [upstream additions](https://github.com/pydata/xarray/pull/2616) to xarray)

In [ ]:
from xbout import open_boutdataset

ds = open_boutdataset('./BOUT.dmp.*.nc', inputfilepath='./BOUT.inp')

In [ ]:
Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset>
Dimensions:   (t: 51, x: 260, y: 1, z: 256)
Dimensions without coordinates: t, x, y, z
Data variables:
    ncalls_i  (t) int32 dask.array<chunksize=(51,), meta=np.ndarray>
    t_array   (t) float64 dask.array<chunksize=(51,), meta=np.ndarray>
    dx        (x, y) float64 dask.array<chunksize=(66, 1), meta=np.ndarray>
    g11       (x, y) float64 dask.array<chunksize=(66, 1), meta=np.ndarray>
    phi       (t, x, y, z) float64 dask.array<chunksize=(51, 66, 1, 256), meta=np.ndarray>
    n         (t, x, y, z) float64 dask.array<chunksize=(51, 66, 1, 256), meta=np.ndarray>
    omega     (t, x, y, z) float64 dask.array<chunksize=(51, 66, 1, 256), meta=np.ndarray>
Attributes:
    metadata:  {'MXG': 2, 'zperiod': 1, 'MZSUB': 256, 'NXPE': 4, 'ZMAX': 1.0,...
    options:   <configparser.ConfigParser object at 0x7f99e7c81860>
Metadata:
{   'BOUT_VERSION': 4.3,
    'MXG': 2,
    'MXSUB': 64,
    'NXPE': 4,}
Options:
<configparser.ConfigParser object at 0x7f99e7c81860>

### Replacement for old collect function

In [ ]:
from xbout import collect

# API matches old boutdata.collect
n = collect('n', path='./', prefix='BOUT.dmp')

# Returns numpy array

Intended to help smooth transition from `numpy` -> `xarray` for users 

### Accessors for BOUT++ specific functionality

"accessors" provided for attaching your own methods to xarray objects

In [ ]:
# Allows for attaching methods without subclassing xarray objects (which would be complex)
ds.bout.do_something_bout_specific()

# Also clearly separates the namespaces

### Plotting tokamak geometries

In [ ]:
ds['psi'].bout.contourf(x='R', y='Z')

<img src="images/xbout_contourf.png" alt="Drawing" style="width: 300px;">

### Animated plots

We wrote methods to animate 1D and 2D plots (by wrapping the animatplot library)

In [ ]:
ds['T'].bout.animate2D(animate_over='time', x='radial', y='binormal', sep_x=sep_x)

![tempgif](T_over_time.gif "segment")

Can even do multiple variables together

In [ ]:
ds.bout.animate_list(['n', 'T', 'vort', 'U', 'T', 'phi'], poloidal_plot=True)

![tempgif](xy-at-0.gif "segment")

### calc module

Accessors would be a good place to build up a libary of analysis functions useful to wider BOUT++ community

### Subclassing accessors for new physics modules

In [ ]:
from xarray import register_dataset_accessor
from xbout.boutdataset import BoutDatasetAccessor

@register_dataset_accessor('storm')
class StormAccessor(BoutAccessor):
    def __init__(self, ds_object):
        super().__init__(ds_object)

    def special_method(self):
        print("Do something only STORM users would want to do")

ds.storm.special_method()

In [ ]:
Do something only STORM users would want to do

### Registering new geometries

Default is to leave `'x', 'y', 'z', 't'` dimensions alone.

Can also choose `geometry='toroidal'`, which adds `R` & `Z` coordinates, and allows poloidal plots

But you can use `register_geometry` to define new geometries, then apply them when loading data

e.g. `geometry='stellarator'` ??

In [ ]:
@register_geometry('stellarator')
def create_stellarator_coordinate(ds):
    # Do some funky 3D maths here
    ...

# Will apply your stellarator coordinate logic internally
ds = open_boutdataset(datapath, geometry='stellarator')

### Example of xarray in use: Analysing parameter scan

Grouping multiple datasets into one xarray object makes it easier to analyse parameter scans

In [ ]:
# Specify folders containing parameter scan
diffusions = ['1e-4', '2e-4', '1e-3', '2e-3', '1e-2', '2e-2', '1e-1', '2e-1']

# Load everything 
file_globs = [file_root + f"mu_n_{diff}/data/BOUT.dmp.*.nc" for diff in diffusions]
datasets = [open_stormdataset(file_glob) for file_glob in file_globs]

# Join it all up
diffusion = xr.DataArray(name='diff', data=diffusions, dims=['diff'])
data = xr.concat(datasets, dim=diffusion)

print(data)

In [ ]:
<xarray.Dataset>
Dimensions:     (binormal: 512, diff: 8, radial: 480, time: 201)
Coordinates:
  * time        (time) float64 0.005548 0.005555 0.005562 ... 0.006923 0.00693
  * binormal    (binormal) float64 0.0 0.000682 0.001364 ... 0.3478 0.3485
  * radial      (radial) float64 0.0 0.0006821 0.001364 ... 0.3254 0.326 0.3267
  * diff        (diff) <U4 '1e-4' '2e-4' '1e-3' '2e-3' ... '2e-2' '1e-1' '2e-1'
Data variables:
    t_array     (diff, time) float64 dask.array<chunksize=(1, 201), meta=np.ndarray>
    dx          (diff, radial) float64 dask.array<chunksize=(1, 10), meta=np.ndarray>
    dy          (diff, radial) float64 dask.array<chunksize=(1, 10), meta=np.ndarray>
    ...

In [ ]:
# Do my power spectrum analysis...
ps = xrft.power_spectrum(data['n'].isel(radial=slice(100,200)), dim=['binormal'], detrend='constant')

# Add a descriptive coordinate for Fourier-transformed axis
k_z_rho_s = ps.coords['freq_binormal'] * data.attrs['params']['rho_s']
ps = ps.assign_coords(k_z_rho_s=k_z_rho_s)

In [ ]:
# Plot absolute power spectrum on log-log against k_perp * rho_s
abs_ps = np.log10(np.abs(ps))
abs_ps = abs_ps.mean(dim=['radial', 'time'])

# Plotting across whole scan
abs_ps.name = 'log10(abs(power spectrum))'
abs_ps.plot(x='k_z_rho_s', xscale='log', hue='diff')   
plt.title('density binormal power spectrum')

<img src="images/power_spectrum.png" alt="Drawing" style="width: 500px;">

# Future Bonus 1: Plans for array duck typing

Currently xarray wraps either `numpy.ndarray` or `dask.array` objects

Possible because they have almost same API, so xarray can select elements in the same way etc.

But what if we extended this idea so xarray could wrap any array-like object with the same API? (so-called "duck-typing")

There are lots of numpy-like arrays in python ecosystem which do clever things:

GPU arrays with `cupy.ndarray`

sparse arrays with `scipy.sparse`

Units with pint, which creates `astropy.units.Quantity` arrays

In [15]:
from astropy import units as u
import numpy as np

np.array([1., 2., 3.]) * u.meter

<Quantity [1., 2., 3.] m>

Plan is to support wrapping any of these with xarray, so you can get benefits of both simultaneously!

e.g. A labelled, unit-aware, GPU-parallelised array in python!!

Support for this is very close - now in master but not officially rolled out.

Open-source so get involved!

Start reading about units support on Github [here](https://github.com/pydata/xarray/issues/525), and sparse arrays [here](https://github.com/pydata/xarray/issues/3213).

# Future Bonus 2: Data analysis software stack using xarray + dask

[Pangeo](https://pangeo.io/index.html) - "A community platform for Big Data geoscience"

<img src="images/pangeo_tech_1.png" alt="Drawing" style="width: 600px;">

(The guy in charge of this project (an [Oceanography professor](https://rabernat.github.io/) at Columbia) has just advertised a full-time software position to support xarray, so will be even more solidly supported in future.)

## Conclusion

- xarray is great

- dask is powerful

- We've made these both accessible to all BOUT++ users

- Solved some common plotting problems

- Structured to be easy to subclass for your pet physics module

- Potential for more additions in future

# Resources

Blog post introducing xarray:
http://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/


xarray GitHub:
https://github.com/pydata/xarray/


xarray documentation:
http://xarray.pydata.org/en/stable/


xarray documentation on dask integration:
http://xarray.pydata.org/en/stable/dask.html


Other useful blogs/tutorials:
http://meteo.unican.es/work/xarray_seminar/xArray_seminar.html
https://rabernat.github.io/research_computing/xarray.html


Useful page from the dask documentation explaining the general idea:
http://docs.dask.org/en/latest/delayed.html

# Secret bonus: xgcm for staggered grids and complex topologies

Interesting work going on in the xgcm package 
https://github.com/xgcm/xgcm

xgcm (Xarray for Global Circulation Models) aims to provide objects which encode complex grids for use with xarray.

Can encode and perform operations on staggered grids:

<img src="./grid2d_hv.svg">

and encode complex topologies by storing connections between different cartesian grids:

<img src="cubed_sphere.jpeg" alt="Drawing" style="width: 400px;">